In [ ]:
!pip install pydriller

# Repository Selection

# Selection Criteria

In [ ]:
from pydriller import Repository
import pandas as pd
import requests

REPO_URL= "https://github.com/Acly/krita-ai-diffusion"

repo = Repository(REPO_URL)
commit_data = []
# for commit in repo.traverse_commits():
#     print(commit)
print(repo)
print(list(repo.traverse_commits()))
repo = Repository(REPO_URL)
commits_data = []
for commit in repo.traverse_commits():
    commits_data.append({
        "hash": commit.hash,
        "msg": commit.msg,
        "parents": [p for p in commit.parents],
        "is_merge": commit.merge,
        "files": [m.filename for m in commit.modified_files]
    })
commits_df = pd.DataFrame(commits_data)
commits_df.to_csv("tf_all_commits.csv", index=False)

[<pydriller.domain.commit.Commit object at 0x7d91384be7e0>, <pydriller.domain.commit.Commit object at 0x7d91384bdcd0>, <pydriller.domain.commit.Commit object at 0x7d91384bc980>, <pydriller.domain.commit.Commit object at 0x7d916495a000>, <pydriller.domain.commit.Commit object at 0x7d9164959640>, <pydriller.domain.commit.Commit object at 0x7d916495b980>, <pydriller.domain.commit.Commit object at 0x7d916495b920>, <pydriller.domain.commit.Commit object at 0x7d916495b0e0>, <pydriller.domain.commit.Commit object at 0x7d916495b9e0>, <pydriller.domain.commit.Commit object at 0x7d916495aab0>, <pydriller.domain.commit.Commit object at 0x7d9164959a00>, <pydriller.domain.commit.Commit object at 0x7d91649591f0>, <pydriller.domain.commit.Commit object at 0x7d9164958500>, <pydriller.domain.commit.Commit object at 0x7d916495ac90>, <pydriller.domain.commit.Commit object at 0x7d91649593d0>, <pydriller.domain.commit.Commit object at 0x7d9164958530>, <pydriller.domain.commit.Commit object at 0x7d9164958ce

In [ ]:
commits_df

,hash,msg,parents,is_merge,files
0,4da6600f5d0563fffe59e8bb3a476e0b7e6d2662,Krita plugin for stable diffusion inpaint work...,[],False,"[.gitattributes, .gitignore, ai_tools.desktop,..."
1,2c31b6464ce773dd0468adf0aee75d7a3a375cac,Use QNetwork for asynchronous http requests wi...,[4da6600f5d0563fffe59e8bb3a476e0b7e6d2662],False,"[diffusion.py, widget.py, workflow.py, conftes..."
2,80ccd7214da5f163a209e9a45a3edf6aa77f55a0,Use asyncio instead of callbacks,[2c31b6464ce773dd0468adf0aee75d7a3a375cac],False,"[__init__.py, diffusion.py, eventloop.py, exte..."
3,10ed9d8b8e1cd1ffbc598c2246f57d54bbfe6436,Tiled upscaling (using ultimate sd upscale scr...,[80ccd7214da5f163a209e9a45a3edf6aa77f55a0],False,"[diffusion.py, widget.py]"
4,aea47b326666dba6dabb6a5c41fee4c03cba5834,Added denoising strength slider (img2img pipel...,[10ed9d8b8e1cd1ffbc598c2246f57d54bbfe6436],False,"[diffusion.py, document.py, image.py, widget.p..."
...,...,...,...,...,...
1313,f95cfe32734e42b462e8e3d1ec751d329645c8f8,Add Nunchaku fp4 models for Blackwell GPUs\n\n...,[5bddaab4598f0b2c6addf4c4d2ff9335fd61e335],False,"[models.json, resources.py, flux-kontext.json,..."
1314,0dd7cbe4ca63dd7ce14bb25e28225b325d5b2c76,Move some things from util.py to new platform....,[f95cfe32734e42b462e8e3d1ec751d329645c8f8],False,"[comfy_client.py, image.py, platform.py, serve..."
1315,21e7183fbbd21735b3511c616b55f4f2da3f1c0b,Add --backend argument for download_models.py\...,[0dd7cbe4ca63dd7ce14bb25e28225b325d5b2c76],False,"[docker.py, download_models.py, test_util.py]"
1316,7140b11e63891ebedb57e7d26b1dd98b6491c1b8,Update Italian translation by @Fabio,[21e7183fbbd21735b3511c616b55f4f2da3f1c0b],False,[it.json]


## Bug Fixing Commit Identification

In [ ]:
# 3. Identify bug-fixing commits
bug_keywords = ['fix', 'bug', 'patch', 'issue', 'error', 'defect', 'resolve', 'correct']
def is_bug_fix(msg):
    return any(word in msg.lower() for word in bug_keywords)
commits_df['is_bugfix'] = commits_df['msg'].apply(is_bug_fix)
bugfix_commits = commits_df[commits_df['is_bugfix']]
bugfix_commits.to_csv("tf_bugfix_commits.csv", index=False)

## Diff Extraction and Analyses:

In [ ]:
bugfix_hashes = bugfix_commits['hash'].tolist()
repo = Repository(REPO_URL, only_commits=bugfix_hashes)

diff_data = []
for commit in repo.traverse_commits():
    for mf in commit.modified_files:
        try:
            diff_data.append({
                "hash": commit.hash,
                "msg": commit.msg,
                "filename": mf.filename,
                "source_before": mf.source_code_before if mf.source_code_before else "",
                "source_after": mf.source_code if mf.source_code else "",
                "diff": mf.diff if mf.diff else "",
            })
        except Exception as e:
            print(f"Skipping {commit.hash}:{mf.filename} due to error: {e}")
            continue

diffs_df = pd.DataFrame(diff_data)
diffs_df.to_csv("tf_bugfix_diffs.csv", index=False)
print("Saved:", diffs_df.shape)


Skipping 90cab405cdb6ff03f01ad884b1d28f8bcc8ed92a:debugpy due to error: SHA b'40a471e24aa9015e2b1a40449a24af3053f7566d' could not be resolved, git returned: b'40a471e24aa9015e2b1a40449a24af3053f7566d missing'
Saved: (1012, 6)


In [ ]:
diffs_df

,hash,msg,filename,source_before,source_after,diff
0,6959b0368460e2787dcd08cd7db7d11e74e87e06,Broke up UI into two phases\nSupport multiple ...,TODO.md,,,
1,6959b0368460e2787dcd08cd7db7d11e74e87e06,Broke up UI into two phases\nSupport multiple ...,diffusion.py,import asyncio\nimport json\nfrom typing impor...,import asyncio\nimport json\nfrom typing impor...,"@@ -7,36 +7,44 @@ from . import settings\n fro..."
2,6959b0368460e2787dcd08cd7db7d11e74e87e06,Broke up UI into two phases\nSupport multiple ...,document.py,import krita\nfrom krita import Krita\nfrom .i...,import krita\nfrom krita import Krita\nfrom .i...,"@@ -16,6 +16,14 @@ class Document:\n @prop..."
3,6959b0368460e2787dcd08cd7db7d11e74e87e06,Broke up UI into two phases\nSupport multiple ...,settings.py,import os \n\n# Images with width/height large...,import os \n\n# Images with width/height large...,"@@ -4,7 +4,7 @@ import os\n max_inpaint_resolu..."
4,6959b0368460e2787dcd08cd7db7d11e74e87e06,Broke up UI into two phases\nSupport multiple ...,widget.py,import asyncio\n\nfrom PyQt5.QtWidgets import ...,from __future__ import annotations\nfrom enum ...,"@@ -1,190 +1,393 @@\n+from __future__ import a..."
...,...,...,...,...,...,...
1007,0977a7228f34080da960115e53121425991e3202,Add generation metadata to saved pngs from the...,settings.py,from __future__ import annotations\nfrom krita...,from __future__ import annotations\nfrom krita...,"@@ -485,6 +485,7 @@ class InterfaceSettings(Se..."
1008,0977a7228f34080da960115e53121425991e3202,Add generation metadata to saved pngs from the...,test_image.py,import pytest\nimport numpy as np\nfrom PyQt5....,import pytest\nimport numpy as np\nimport stru...,"@@ -1,5 +1,8 @@\n import pytest\n import numpy..."
1009,0977a7228f34080da960115e53121425991e3202,Add generation metadata to saved pngs from the...,test_text.py,"from ai_diffusion.text import merge_prompt, ex...",from ai_diffusion.text import (\n merge_pro...,"@@ -1,6 +1,14 @@\n-from ai_diffusion.text impo..."
1010,7c636f7e5648ee9c23da0e9a94d8261b1aaa224a,Log more detailed error when custom workflow i...,custom_workflow.py,import asyncio\nimport json\nimport re\n\nfrom...,import asyncio\nimport json\nimport re\n\nfrom...,"@@ -177,6 +177,7 @@ class WorkflowCollection(Q..."


In [ ]:
!pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model locally
tokenizer = AutoTokenizer.from_pretrained("mamiksik/CommitPredictorT5")
model = AutoModelForSeq2SeqLM.from_pretrained("mamiksik/CommitPredictorT5")

import torch

def llm_infer_local(diff_text):
    # prompt = f"Write a concise commit message describing the following code diff:\n{diff_text}"
#     prompt = f"""
# Summarize this diff. Focus on *what changed* (e.g., fix, add, remove, rename) and *where* (file/function names).

# Diff:
# {diff_text}
# """
    inputs = tokenizer(diff_text, return_tensors="pt", truncation=True)
    output_ids = model.generate(**inputs, max_length=96)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Example usage:
# Apply to your DataFrame (assuming diffs_df with column 'diff')
diffs_df['llm_message'] = diffs_df['diff'].apply(llm_infer_local)
diffs_df.to_csv("algos_bugfix_with_llm.csv", index=False)

def is_rectified(orig, llm):
    return orig.strip().lower() != llm.strip().lower() if llm else False

diffs_df['rectified'] = diffs_df.apply(
    lambda row: is_rectified(row['msg'], row['llm_message']), axis=1
)
diffs_df.to_csv("algos_rectified_bugfixes.csv", index=False)


In [ ]:
diffs_df

,hash,msg,filename,source_before,source_after,diff,llm_message,rectified
0,6959b0368460e2787dcd08cd7db7d11e74e87e06,Broke up UI into two phases\nSupport multiple ...,TODO.md,,,,fix bug in test-http-client,True
1,6959b0368460e2787dcd08cd7db7d11e74e87e06,Broke up UI into two phases\nSupport multiple ...,diffusion.py,import asyncio\nimport json\nfrom typing impor...,import asyncio\nimport json\nfrom typing impor...,"@@ -7,36 +7,44 @@ from . import settings\n fro...",add tests for the rest of the qnet api,True
2,6959b0368460e2787dcd08cd7db7d11e74e87e06,Broke up UI into two phases\nSupport multiple ...,document.py,import krita\nfrom krita import Krita\nfrom .i...,import krita\nfrom krita import Krita\nfrom .i...,"@@ -16,6 +16,14 @@ class Document:\n @prop...",add missing docutils methods,True
3,6959b0368460e2787dcd08cd7db7d11e74e87e06,Broke up UI into two phases\nSupport multiple ...,settings.py,import os \n\n# Images with width/height large...,import os \n\n# Images with width/height large...,"@@ -4,7 +4,7 @@ import os\n max_inpaint_resolu...",add batch size to the debug image,True
4,6959b0368460e2787dcd08cd7db7d11e74e87e06,Broke up UI into two phases\nSupport multiple ...,widget.py,import asyncio\n\nfrom PyQt5.QtWidgets import ...,from __future__ import annotations\nfrom enum ...,"@@ -1,190 +1,393 @@\n+from __future__ import a...",add missing docstrings,True
...,...,...,...,...,...,...,...,...
1007,0977a7228f34080da960115e53121425991e3202,Add generation metadata to saved pngs from the...,settings.py,from __future__ import annotations\nfrom krita...,from __future__ import annotations\nfrom krita...,"@@ -485,6 +485,7 @@ class InterfaceSettings(Se...",add language switch,True
1008,0977a7228f34080da960115e53121425991e3202,Add generation metadata to saved pngs from the...,test_image.py,import pytest\nimport numpy as np\nfrom PyQt5....,import pytest\nimport numpy as np\nimport stru...,"@@ -1,5 +1,8 @@\n import pytest\n import numpy...",add test for save_png_w_itxt,True
1009,0977a7228f34080da960115e53121425991e3202,Add generation metadata to saved pngs from the...,test_text.py,"from ai_diffusion.text import merge_prompt, ex...",from ai_diffusion.text import (\n merge_pro...,"@@ -1,6 +1,14 @@\n-from ai_diffusion.text impo...",add tests for the new `imaging` language,True
1010,7c636f7e5648ee9c23da0e9a94d8261b1aaa224a,Log more detailed error when custom workflow i...,custom_workflow.py,import asyncio\nimport json\nimport re\n\nfrom...,import asyncio\nimport json\nimport re\n\nfrom...,"@@ -177,6 +177,7 @@ class WorkflowCollection(Q...",add missing import,True


## Rectifier Formulation

In [ ]:
import re
import os

def regex_rectifier(diff_text):
    """
    Rule-based rectifier that summarizes changes by file, type, and keywords.
    """
    if not isinstance(diff_text, str) or not diff_text.strip():
        return "Code changes"

    # Extract file paths
    files = re.findall(r'diff --git a/(\S+)', diff_text)
    added_lines = re.findall(r'^\+[^+].*', diff_text, flags=re.MULTILINE)
    removed_lines = re.findall(r'^-[^-].*', diff_text, flags=re.MULTILINE)

    message_parts = []

    # --- File summary ---
    if files:
        if len(files) == 1:
            ext = os.path.splitext(files[0])[1].lstrip(".")
            message_parts.append(f"Update {files[0]}" if ext == "" else f"Update {ext} file {files[0]}")
        elif len(files) <= 3:
            listed = ", ".join([os.path.basename(f) for f in files])
            message_parts.append(f"Update {len(files)} files ({listed})")
        else:
            message_parts.append(f"Update {len(files)} files")

    # --- Change type ---
    if added_lines and not removed_lines:
        message_parts.append("added code")
    elif removed_lines and not added_lines:
        message_parts.append("removed code")
    elif added_lines and removed_lines:
        message_parts.append("modified code")

    # --- Function/class mentions ---
    functions = re.findall(r'^\+.*\bdef (\w+)\b', diff_text, flags=re.MULTILINE)
    classes   = re.findall(r'^\+.*\bclass (\w+)\b', diff_text, flags=re.MULTILINE)
    if functions:
        message_parts.append("added/changed functions: " + ", ".join(functions[:3]) + ("..." if len(functions) > 3 else ""))
    if classes:
        message_parts.append("added/changed classes: " + ", ".join(classes[:3]) + ("..." if len(classes) > 3 else ""))

    # --- Keyword hints ---
    keywords = {
        "bug": "bug fix",
        "error": "fix error handling",
        "test": "update tests",
        "doc": "update documentation",
        "readme": "update README",
        "log": "improve logging",
        "refactor": "refactor code",
    }
    diff_lower = diff_text.lower()
    for k, v in keywords.items():
        if k in diff_lower:
            message_parts.append(v)
            break  # only add one for clarity

    # --- Line counts ---
    if added_lines or removed_lines:
        message_parts.append(f"(+{len(added_lines)} -{len(removed_lines)} lines)")

    # --- Final assembly ---
    if not message_parts:
        return "Code changes"
    return "; ".join(message_parts)
diffs_df['rectified_message'] = diffs_df['diff'].apply(regex_rectifier)
diffs_df.to_csv("algos_regex_rectified_bugfixes.csv", index=False)

def add_filename_to_message(row):
    """
    Post-process rectified message to include the filename.
    """
    msg = row['rectified_message']
    filename = str(row['filename']) if pd.notna(row['filename']) else "unknown"
    return f"{msg}; in {filename}"

# Apply post-processing
diffs_df['rectified_message'] = diffs_df.apply(add_filename_to_message, axis=1)
diffs_df.to_csv("algos_regex_rectified_bugfixes_with_files.csv", index=False)


## Evaluation: Research Questions

In [ ]:
import re

import re

def is_precise(message, diff_text):
    """
    Rule-based check if a commit message is precise.
    A good message should:
    - Mention file/module/function/class being changed
    - Contain an action verb
    - Contain a domain/context keyword
    - Avoid vague/empty phrasing
    """
    if not message or not message.strip():
        return False

    msg = message.lower().strip()

    # --- Rule 0: reject vague boilerplate ---
    vague_patterns = [
        r"\bfix(es|ed)?\b$", r"\bupdate(s|d)?\b$", r"minor change(s)?",
        r"misc", r"temp", r"wip", r"stuff", r"changes"
    ]
    if any(re.search(p, msg) for p in vague_patterns):
        return False

    # --- Extract files and identifiers from diff ---
    files = re.findall(r'diff --git a/(\S+)', diff_text.lower())
    functions = re.findall(r'\bdef (\w+)\b', diff_text.lower())
    classes = re.findall(r'\bclass (\w+)\b', diff_text.lower())

    # --- Rule 1: Mention file/module/function/class ---
    mentions_code = any(
        token in msg
        for token in [f.split("/")[-1].split(".")[0] for f in files] + functions + classes
    )
    if mentions_code:
        return True

    # --- Rule 2: Action verbs ---
    actions = ["fix", "add", "remove", "update", "refactor", "rename", "implement", "improve"]
    has_action = any(a in msg for a in actions)

    # --- Rule 3: Domain/context keywords ---
    domains = ["bug", "error", "test", "doc", "readme", "parser", "api", "log", "config", "build"]
    has_domain = any(d in msg for d in domains)

    # --- Rule 4: Good if action + domain present ---
    if has_action and has_domain:
        return True

    # --- Default fallback ---
    return False


# RQ1: Developer eval
diffs_df['dev_precise'] = diffs_df.apply(
    lambda row: is_precise(row['msg'], row['diff']), axis=1
)

# RQ2: LLM eval
diffs_df['llm_precise'] = diffs_df.apply(
    lambda row: is_precise(row['llm_message'], row['diff']), axis=1
)

# RQ3: Rectifier eval
diffs_df['rectified_precise'] = diffs_df.apply(
    lambda row: is_precise(row['rectified_message'], row['diff']), axis=1
)

# Hit rates
rq1_hit = diffs_df['dev_precise'].mean()
rq2_hit = diffs_df['llm_precise'].mean()
rq3_hit = diffs_df['rectified_precise'].mean()

print("RQ1 (Developer hit rate):", rq1_hit)
print("RQ2 (LLM hit rate):", rq2_hit)
print("RQ3 (Rectifier hit rate):", rq3_hit)


RQ1 (Developer hit rate): 0.43478260869565216
RQ2 (LLM hit rate): 0.4990118577075099
RQ3 (Rectifier hit rate): 0.7035573122529645
